## Load images and labels

In [1]:
from scipy.misc import imread, imresize
import os
import numpy as np

def predict_batch(folder, img_size=None):
    img_list = []
    label_list = []

    for filename in os.listdir(folder):
        
        if filename.endswith(".jpg"):
            img = imread(os.path.join(folder,filename))
            lb = filename.split('_')[1]
            lb = lb.lower()
        
        if img_size:
            img = imresize(img,img_size)

        img = img.astype('float32')
        
        # convert image to greyscale
        img = img.sum(axis=2) / 3.
        img /= np.std(img)
        
        img = img[:, :, np.newaxis]
        
        img_list.append(img)
        label_list.append(lb)
    try:
        img_batch = np.stack(img_list, axis=0)
    except:
        raise ValueError(
            'when both img_size and crop_size are None, all images '
            'in image_paths must have the same shapes.')

    return img_batch, label_list 


In [2]:
YOUR_FOLDER_NAME = "Synth90k"

output, labels = predict_batch(YOUR_FOLDER_NAME, (32, 100))
print("output shape", output.shape)
print ('label shape', len(labels))

output shape (8092, 32, 100, 1)
label shape 8092


## Conv-net model

The base CNN has five convolutional layers and two fully connected layers.


In [3]:
from keras import models
from keras.layers import Flatten, Dropout, MaxPooling2D,Activation
from keras.layers import Dense, InputLayer, Convolution2D
import tensorflow as tf

def Ngram_CNN(n_classes):
    model = models.Sequential()

    # input layer
    model.add(InputLayer(input_shape=(32,100,1)))

    # first convolutional layer
    model.add(Convolution2D(64, 5, 5, activation='relu', subsample=(1, 1),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


    # second convolutional layer
    model.add(Convolution2D(128, 5, 5, activation='relu',subsample=(1, 1),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


    # third convolutional layer
    model.add(Convolution2D(256, 3, 3, activation='relu',subsample=(1, 1),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


    # fourth convolutional layer
    model.add(Convolution2D(512, 3, 3, activation='relu',subsample=(1, 1),border_mode='same'))


    # fifth convolutional layer
    model.add(Convolution2D(512, 3, 3, activation='relu', subsample=(1, 1),border_mode='same')) 


    # first fully connected layer
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu')) 


    # second fully connected layer
    model.add(Dropout(0.5))
    model.add(Dense(4096))

    # N_gram classes layer
    model.add(Dropout(0.5))
    model.add(Dense(n_classes))
    model.add(Activation('sigmoid'))
    
    model.compile(loss ='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

Using TensorFlow backend.


## N-gram building

In [4]:
from func_util_Ngram import ground_truth

ngramDict, y_gt = ground_truth(labels,N=4)

## train model

In [5]:
X_train, y_train = output, np.array(y_gt)

# train model
batch_size = 32
nb_epoch = 10

model = Ngram_CNN(len(ngramDict))

model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,validation_split=0.1)

ResourceExhaustedError: OOM when allocating tensor with shape[4096,19064]
	 [[Node: Variable_40/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_40"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_40, Const_45)]]

Caused by op 'Variable_40/Assign', defined at:
  File "/home/yunji/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yunji/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yunji/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/yunji/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 653, in launch_instance
    app.start()
  File "/home/yunji/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/yunji/anaconda3/lib/python3.5/site-packages/z

In [11]:
model.save_weights("weignts_ngram.h5")